In [2]:

                        #This code block can be ignored
#General outline of the newer updated charge balance equation (model in NLSF minimiser)

model = (self.V0 + self.Va + dataframe["m"])*(dataframe["H"] - dataframe["OH"])
        - ((self.V0 + self.Va) * self.H0) 
        + (dataframe["m"] * C_NaOH)

        - V0 *(BT / (1 + H / KB)) # optional, but used for all real seawater samples. BT and KB calculated 
        - V0 * (PT / (1 + H / KP2))  # optional, used if PT is above a certain value. PT user supplied, KP2 calculated 
        - V0 * (SiT / (1 + H / KSi)) # optional, used if SiT is above a certain value. SiT user supplied, KSi calculated 

        - (Vb * CTNa) / (1 + (H / Kc1) + (Kc2 / H)) #K1c and K2c are carbonate system dissociation constants 
        - (2 * Vb * CTNa) / (1 + (H**2 / (Kc1 * Kc2)) + (H / Kc2)) #See note below


        - (self.V0)*(X1/(1+dataframe["H"]/K_X1))
        - (self.V0)*(X2/(1+dataframe["H"]/K_X2))
        - (self.V0)*(X3/(1+dataframe["H"]/K_X3))
        


#CTNa is a user supplier value relating to the carbonate content of the NaOH solution used for the titration.
# An intial estimate of CTNa is provided, it is parameter which is varied along with H0, X1 and K_X1 is minimiser001
# Minner001 pH range 0 - 5. X1, K_X1, H0 and CTNa are varied in the NLSF. initial values for X1, K_X1 and CTNa are found. Final value for H0 is found.
# Minner002 pH range 0 - pHmax*0.80.  H0 is fixed.  X1, K_X1 and CTNa are varied. Final values for X1, K_X1 are found. 
# Minner003 pH range full.  H0 X1, K_X1 are fixed. X2, K_X2 and CTNa varied. Final values for X2, K_X2 and CTNa are found. 
# Minner004 H0 X1, K_X1, X2, K_X2 and CTNa are fixed. X3 and K_X3 are varied. Final values for X3, K_X3 are found. 
                                                                   



IndentationError: unexpected indent (2277741218.py, line 2)

In [ ]:
        #Updated Minimiser

#I have adapted the charge balance equations, e.g. model in minimiser_no == X, to include a term to account for the carbonate present in the NaOH titre, CTNa (Carbonate Total NaOH)
#The user should be given the option to use the CTNA term or not, eg user doesnt want to use it CTNa = 0
#One issue I can see that may arise is how the CTNa term reacts when being = 0 in the minimiser - you will probably have a better idea of how to work this
#I have changed the nomencleture so it should be in line with what you are already using - I may have missed some minor things.
#I have left the carbonate system dissociation constants as Kc1 and Kc2 as they are chosen specifically by the user - see the code block below for detailed description
    


def minimise(self,minimiser_no):
        if minimiser_no < 3:
            dataframe = self.cleaned_df_NaOH
        elif 2 < minimiser_no < 5:
            dataframe = self.df_NaOH
        elif minimiser_no > 4: 
            raise ValueError("minimiser_no must be in the range 1-4")

        minimiser_2_UL = ( np.max(dataframe['pH'])*0.80 ) #minimiser 2 upper limit pH max * 0.80 


        if minimiser_no == 1:
            dataframe = dataframe[dataframe["pH"].between(0,5)]
        elif minimiser_no == 2:
            dataframe = dataframe[dataframe["pH"].between(0,minimiser_2_UL)]

        x = dataframe["m"]
        data = dataframe["H"]

        if minimiser_no == 1:
            def fcn2min(params, x, data):
                H0 = params['H0']
                C_NaOH = params['C_NaOH']
                CTNa = #((user supplied NaOH CO_3^2- value [originally µmol.kg-1] )*10^-6)/1000 [now in moles/mL] * Volume of base Vb 
                       # this results in the CO_3^2- added per the amount of base used in the titration 
                X1 = params['X1']
                K_X1 = params['K_X1']


                model = ((self.V0 + self.Va+ dataframe["m"])*(dataframe["H"]-dataframe["OH"]) 
                         -((self.V0+self.Va)*H0)
                         +(dataframe["m"]*C_NaOH) 

                         - self.V0 *(BT / (1 + dataframe["H"] / dataframe["KB"])) # optional, but used for all real seawater samples. BT and KB calculated. 
                         - self.V0 * (PT / (1 + dataframe["H"] / dataframe["KP2"]))  # optional, used if PT is above a certain value. PT user supplied, KP2 calculated 
                         - self.V0 * (SiT / (1 + dataframe["H"] / dataframe["KSi"])) # optional, used if SiT is above a certain value. SiT user supplied, KSi calculated 

                         - (self.Vb * self.CTNa) / (1 + (dataframe["H"] / Kc1) + (Kc2 / dataframe["H"])) #K1c and K2c are carbonate system dissociation constants 
                         - (2 * self.Vb * self.CTNa) / (1 + (dataframe["H"]**2 / (Kc1 * Kc2)) + (dataframe["H"] / Kc2)) #K1c and K2c chosen by the user 

                         - (self.V0)*(X1/(1+dataframe["H"]/K_X1)))
              
                return model - data
        elif minimiser_no == 2:
            def fcn2min(params, x, data):
                H0 = params['H0']
                C_NaOH = params['C_NaOH']
                CTNa = #((user supplied NaOH CO_3^2- value [originally µmol.kg-1] )*10^-6)/1000 [now in moles/mL] * Volume of base Vb 
                       # this results in the CO_3^2- added per the amount of base used in the titration usually on scale of 10**-9
                X1 = params['X1']
                K_X1 = params['K_X1']
                X2 = params['X2']
                K_X2 = params['K_X2']

                model = ((self.V0 + self.Va+ dataframe["m"])*(dataframe["H"]-dataframe["OH"]) 
                         -((self.V0+self.Va)*H0)
                         +(dataframe["m"]*C_NaOH) 

                         - self.V0 *(BT / (1 + dataframe["H"] / dataframe["KB"])) # optional, but used for all real seawater samples. BT and KB calculated 
                         - self.V0 * (PT / (1 + dataframe["H"] / dataframe["KP2"]))  # optional, used if PT is above a certain value. PT user supplied, KP2 calculated 
                         - self.V0 * (SiT / (1 + dataframe["H"] / dataframe["KSi"])) # optional, used if SiT is above a certain value. SiT user supplied, KSi calculated 

                         - (self.Vb * self.CTNa) / (1 + (dataframe["H"] / Kc1) + (Kc2 / dataframe["H"])) #K1c and K2c are carbonate system dissociation constants 
                         - (2 * self.Vb * self.CTNa) / (1 + (dataframe["H"]**2 / (Kc1 * Kc2)) + (dataframe["H"] / Kc2)) #K1c and K2c chosen by the user 

                         - (self.V0)*(X1/(1+dataframe["H"]/K_X1))
                         - (self.V0)*(X2/(1+dataframe["H"]/K_X3)))
              
                return model - data
        elif minimiser_no == 3:
                H0 = params['H0']
                C_NaOH = params['C_NaOH']
                CTNa = #((user supplied NaOH CO_3^2- value [originally µmol.kg-1] )*10^-6)/1000 [now in moles/mL] * Volume of base Vb 
                       # this results in the CO_3^2- added per the amount of base used in the titration usually on scale of 10**-9
                X1 = params['X1']
                K_X1 = params['K_X1']
                X2 = params['X2']
                K_X2 = params['K_X2']
                X3 = params['X']
                K_X3 = params['K_X3']

                model = ((self.V0 + self.Va+ dataframe["m"])*(dataframe["H"]-dataframe["OH"]) 
                         -((self.V0+self.Va)*H0)
                         +(dataframe["m"]*C_NaOH) 

                         - self.V0 *(BT / (1 + dataframe["H"] / dataframe["KB"])) # optional, but used for all real seawater samples. BT and KB calculated 
                         - self.V0 * (PT / (1 + dataframe["H"] / dataframe["KP2"]))  # optional, used if PT is above a certain value. PT user supplied, KP2 calculated 
                         - self.V0 * (SiT / (1 + dataframe["H"] / dataframe["KSi"])) # optional, used if SiT is above a certain value. SiT user supplied, KSi calculated 

                         - (self.Vb * self.CTNa) / (1 + (dataframe["H"] / Kc1) + (Kc2 / dataframe["H"])) #K1c and K2c are carbonate system dissociation constants 
                         - (2 * self.Vb * self.CTNa) / (1 + (dataframe["H"]**2 / (Kc1 * Kc2)) + (dataframe["H"] / Kc2)) #K1c and K2c chosen by the user 

                         - (self.V0)*(X1/(1+dataframe["H"]/K_X1))
                         - (self.V0)*(X2/(1+dataframe["H"]/K_X2))
                         - (self.V0)*(X3/(1+dataframe["H"]/K_X3)))
              
                return model - data
        elif minimiser_no == 4:
                H0 = params['H0']
                C_NaOH = params['C_NaOH']
                CTNa = #((user supplied NaOH CO_3^2- value [originally µmol.kg-1] )*10^-6)/1000 [now in moles/mL] * Volume of base Vb 
                       # this results in the CO_3^2- added per the amount of base used in the titration usually on scale of 10**-9
                X1 = params['X1']
                K_X1 = params['K_X1']
                X2 = params['X2']
                K_X2 = params['K_X2']
                X3 = params['X3']
                K_X3 = params['K_X3']
                X4 = params['X4']
                K_X4 = params['K_X4']

                model = ((self.V0 + self.Va+ dataframe["m"])*(dataframe["H"]-dataframe["OH"]) 
                         -((self.V0+self.Va)*H0)
                         +(dataframe["m"]*C_NaOH) 

                         - self.V0 *(BT / (1 + dataframe["H"] / dataframe["KB"])) # optional, but used for all real seawater samples. BT and KB calculated 
                         - self.V0 * (PT / (1 + dataframe["H"] / dataframe["KP2"]))  # optional, used if PT is above a certain value. PT user supplied, KP2 calculated 
                         - self.V0 * (SiT / (1 + dataframe["H"] / dataframe["KSi"])) # optional, used if SiT is above a certain value. SiT user supplied, KSi calculated 

                         - (self.Vb * self.CTNa) / (1 + (dataframe["H"] / Kc1) + (Kc2 / dataframe["H"])) #K1c and K2c are carbonate system dissociation constants 
                         - (2 * self.Vb * self.CTNa) / (1 + (dataframe["H"]**2 / (Kc1 * Kc2)) + (dataframe["H"] / Kc2)) #K1c and K2c chosen by the user 

                         - (self.V0)*(X1/(1+dataframe["H"]/K_X1))
                         - (self.V0)*(X2/(1+dataframe["H"]/K_X2))
                         - (self.V0)*(X3/(1+dataframe["H"]/K_X3))
                         - (self.V0)*(X4/(1+dataframe["H"]/K_X4)))
                return model - data

        params = Parameters()
        self.add_params(params,minimiser_no)
    
        # do fit, here with leastsq model
        minner = Minimizer(fcn2min, params, fcn_args=(x, data))
        kws  = {'options': {'maxiter':100}} # Add this to minner at some point

        result = minner.minimize()
        self.get_params(result,minimiser_no)

In [ ]:
            #Updated SSR
    #I have included the new CTNa term into the "m_calc_00X" equations.
    #I have changed the nomencleture so it should be in line with what you are already using.
    #I have left the carbonate system dissociation constants as Kc1 and Kc2 as they are chosen specifically by the user - see the next code block for detailed description
    
    
    
    def ssr(self,minimiser_no):
        cleaned_dataframe = self.cleaned_df_NaOH
        dataframe = self.df_NaOH
        if minimiser_no == 1:
            cleaned_dataframe["m_calc_001"] = ( -((self.V0*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]) 
                                      - self.H0*(self.V0+self.Va)
                                      - (self.BT*self.V0)/((cleaned_dataframe["H"]/dataframe["KB"])+1) 
                                      - (self.PT*self.V0)/((cleaned_dataframe["H"]/dataframe["KP2"])+1) 
                                      - (self.SiT*self.V0)/((cleaned_dataframe["H"]/dataframe["KSi"])+1) 
                                      - (self.X1*self.V0)/((cleaned_dataframe["H"]/self.K_X1)+1) 
                                      + self.Va*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]))
                                      / ((2*self.CTNa)/((cleaned_dataframe["H"]**2/( Kc1* Kc2))+(cleaned_dataframe["H"]/Kc2)+1)
                                      + self.CTNa/((cleaned_dataframe["H"]/ Kc1)+(Kc2/cleaned_dataframe["H"])+1)
                                      -cleaned_dataframe["H"] + cleaned_dataframe["OH"] + self.C_NaOH)) )
    
    
                                     
            SSR = np.sum((cleaned_dataframe['m']-cleaned_dataframe["m_calc_001"])**2)

        elif minimiser_no == 2:
            cleaned_dataframe["m_calc_002"] = ( -((self.V0*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]) 
                                      - self.H0*(self.V0+self.Va)
                                      - (self.BT*self.V0)/((cleaned_dataframe["H"]/dataframe["KB"])+1) 
                                      - (self.PT*self.V0)/((cleaned_dataframe["H"]/dataframe["KP2"])+1) 
                                      - (self.SiT*self.V0)/((cleaned_dataframe["H"]/dataframe["KSi"])+1) 
                                      - (self.X1*self.V0)/((cleaned_dataframe["H"]/self.K_X1)+1) 
                                      - (self.X2*self.V0)/((cleaned_dataframe["H"]/self.K_X2)+1)
                                      + self.Va*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]))
                                      / ((2*self.CTNa)/((cleaned_dataframe["H"]**2/( Kc1* Kc2))+(cleaned_dataframe["H"]/Kc2)+1)
                                      + self.CTNa/((cleaned_dataframe["H"]/ Kc1)+(Kc2/cleaned_dataframe["H"])+1)
                                      -cleaned_dataframe["H"] + cleaned_dataframe["OH"] + self.C_NaOH)) )
    
    
                                     
            SSR = np.sum((cleaned_dataframe['m']-cleaned_dataframe["m_calc_002"])**2)

        elif minimiser_no == 3:
            cleaned_dataframe["m_calc_003"] = ( -((self.V0*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]) 
                                      - self.H0*(self.V0+self.Va)
                                      - (self.BT*self.V0)/((cleaned_dataframe["H"]/dataframe["KB"])+1) 
                                      - (self.PT*self.V0)/((cleaned_dataframe["H"]/dataframe["KP2"])+1) 
                                      - (self.SiT*self.V0)/((cleaned_dataframe["H"]/dataframe["KSi"])+1) 
                                      - (self.X1*self.V0)/((cleaned_dataframe["H"]/self.K_X1)+1) 
                                      - (self.X2*self.V0)/((cleaned_dataframe["H"]/self.K_X2)+1)
                                      - (self.X3*self.V0)/((cleaned_dataframe["H"]self./K_X3)+1)
                                      + self.Va*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]))
                                      / ((2*self.CTNa)/((cleaned_dataframe["H"]**2/( Kc1* Kc2))+(cleaned_dataframe["H"]/Kc2)+1)
                                      + self.CTNa/((cleaned_dataframe["H"]/ Kc1)+(Kc2/cleaned_dataframe["H"])+1)
                                      -cleaned_dataframe["H"] + cleaned_dataframe["OH"] + self.C_NaOH)) )
    
    
                                     
            SSR = np.sum((cleaned_dataframe['m']-cleaned_dataframe["m_calc_003"])**2)


        elif minimiser_no == 4:
            cleaned_dataframe["m_calc_004"] = ( -((self.V0*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]) 
                                      - self.H0*(self.V0+self.Va)
                                      - (self.BT*self.V0)/((cleaned_dataframe["H"]/dataframe["KB"])+1) 
                                      - (self.PT*self.V0)/((cleaned_dataframe["H"]/dataframe["KP2"])+1) 
                                      - (self.SiT*self.V0)/((cleaned_dataframe["H"]/dataframe["KSi"])+1) 
                                      - (self.X1*self.V0)/((cleaned_dataframe["H"]/self.K_X1)+1) 
                                      - (self.X2*self.V0)/((cleaned_dataframe["H"]/self.K_X2)+1)
                                      - (self.X3*self.V0)/((cleaned_dataframe["H"]self./K_X3)+1)
                                      + self.Va*(cleaned_dataframe["H"]-cleaned_dataframe["OH"]))
                                      / ((2*self.CTNa)/((cleaned_dataframe["H"]**2/( Kc1* Kc2))+(cleaned_dataframe["H"]/Kc2)+1)
                                      + self.CTNa/((cleaned_dataframe["H"]/ Kc1)+(Kc2/cleaned_dataframe["H"])+1)
                                      -cleaned_dataframe["H"] + cleaned_dataframe["OH"] + self.C_NaOH)) )
    
    
                                     
            SSR = np.sum((cleaned_dataframe['m']-cleaned_dataframe["m_calc_004"])**2)
        return SSR 

In [30]:


                            #Dissociation constants for phosphoric acid, silicic acid. The user has no choice with these.

#import numpy as np
# Simulated temperature, salinity and ionic strength to proof check equations
T=298.15
S=35
ImO = (19.924*S)/(1000-1.005*S)


#dissociation for phosphoric acid from Millero1995 
KP2 = -8814.715/T + 172.0883 - 27.927*np.log(T) + (-160.34/T + 1.3566)*S**0.5 + (0.37335/T - 0.05778)*S 
#dissociation for silicic acid from Millero1995
KSi = -8904.2/T + 117.385 - 19.334*np.log(T) +(-458.79/T + 3.5913)*(ImO)**0.5 + (188.74/T - 1.5998)*(ImO) + (-12.1652/T + 0.07871)*(ImO)**2 + np.log(1-0.001005*S)



                            #Dissociation constants for carbonate system Kc1 and Kc2. The user has a choice as to what constants they use.
                            #There are several options, Cai1998, Prieto-Mojica2002, Waters2013
                            #NOTE: the K values are used in all equtions not pKa values. 


#pK1, pK2 from Cai1998, valid S = 0 - 40 
#These K1 K2 values have large uncertainty associated with them are not considered for open ocean use (Woosley2021)
F1T = 200.1/T + 0.3220
F2T = -129.24/T + 1.4381
pK1_Cai = 3404.71/T + 0.032786*T - 14.8435 - 0.071692*F1T*(S**0.5) + 0.0021487*S
pK2_Cai = 2902.39/T + 0.02379*T - 6.4980 - 0.3191*F2T*(S**0.5) + 0.0198*S
K1_Cai = 10.0 ** -pK1_Cai
K2_Cai = 10.0 ** -pK2_Cai

#pK1, pK2 from Prieto-Mojica2002, valid S = 5 - 42 
pK1_Prieto = -43.6977 - 0.0129037*S + (1.364*10**-4)*(S**2) + 2885.378/T +7.045159*np.log(T)
pK2_Prieto = -452.0940 + (13.142162*S)-(8.101*10**-4)*S**2 + 21263.61/T + 68.483143*np.log(T) + (-581.4428*S + 0.259601*S**2)/T - 1.967035*S*np.log(T)
K1_Prieto = 10 ** -pK1_Prieto
K2_Prieto = 10 ** - pK2_Prieto

#pK1, pK2 Waters2013, valid S 4 - 45

pK01 = -126.34048 + 6320.813/T + 19.568224*np.log(T)
pK02 = -90.18333 + 5143.692/T + 14.61358*np.log(T)
pK1_Waters = pK01 + 13.568513*S**0.5 + 0.031645*S + (-5.3834*10**-5)*S**2 + (-539.2304*S**0.5)/T + (-5.635*S/T) + (-2.0901396*(S**0.5*np.log(T)))
pK2_Waters = pK02 + 21.389248*S**0.5 + 0.12452358*S + (-3.7447*10**-4)*S**2 + (-787.3736*S**0.5)/T + (-19.84233*S/T) + (-3.3773006*(S**0.5*np.log(T)))
K1_Waters = 10.0 ** -pK1_Waters
K2_Waters = 10.0 ** -pK2_Waters


#Add Millero2010



#Checks to make sure equations are giving correct answer
print(KP2) #correct answer is: -13.727
print(KSi) #correct answer is: -21.61
print(pK1_Cai) #correct answer is: 6.00
print(pK2_Cai) #correct answer is: 9.12
print(pK1_Prieto) #no correct answer supplied
print(pK2_Prieto) #no correct answer supplied
print(pK1_Waters) #correct answer 5.8510
print(pK2_Waters) #correct answer 8.9768
print(K1_Waters) #correct answer 5.8510
print(K2_Waters) #correct answer 8.9768




-13.727467952937484
-21.606962839691953
6.005078229891063
9.126096411637485
5.835840511327831
8.949810219084668
5.851020355397409
8.978069180666381
1.409222746555018e-06
1.051794315563975e-09


(6.005078229891063, 9.126096411637485)